In [1]:
"""
교차 검증과 그리드 서치

검증 세트

이 방법은 너무 단순함 - 실제로 많이 사용하는 방법임

훈련세트 약60%, 검증세트 약 20%, 테스트세트 약 20%

훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가함 
이런 식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고름
그다음 이 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 
모델을 다시 훈련함
그리고 마지막에서 테스트 세트에서 최종 점수를 평가함

"""
import pandas as pd
wine = pd.read_csv("https://bit.ly/wine_csv_data")

In [2]:
"""
class 열을 타깃으로 사용
나머지 열은 특성 배열에 저장
"""

data = wine[["alcohol", "sugar", "pH"]].to_numpy()
target = wine["class"].to_numpy()

In [3]:
"""
훈련 세트와 테스트 세트로 나눔

"""
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
data, target, test_size=0.2, random_state =42)

In [4]:
"""
훈련 세트 선택
train_input과 train_target을 다시 train_test_split()함수에 넣어 
훈련 세트 sub_input, sub_target과 검증 세트 val_input, val_target을 만듬
"""
sub_input, val_input, sub_target, val_target = train_test_split(
train_input, train_target, test_size = 0.2, random_state=42)

In [5]:
"""
train_test_split() 함수를 2번 적용해서 
훈련 세트와 검증 세트로 나눈것
"""
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
"""
sub_input, sub_target과 val_input, val_target을 사용해
모델을 만들고 평가
"""
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [7]:
"""
교차 검증
검증 세트를 만드느라 훈련세트가 줄었다
교차 검증을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다

3-폴드 교차 검증
k-폴드 교차 검증 이라고 하며 훈련 세트를 몇 부분으로 나누나에 따라 다르게 부름
k-겹 교차 검증 이라고도 함

예시를 위헤 3-폴드 교차 검증을 예시로 들었지만
보통 5-폴드 교차 검증이나 10-폴드 교차검증을 많이 사용함
이렇게 하면 데이터의 80~90%까지 훈련에 사용할 수 있다
검증세트가 줄어들지만 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수로 
생각할 수 있다

사이킷런에 cross_validate()라는 교차 검증 함수가 있다
사용법은 간단한데 
먼저 평가할 모델 객체를 첫 번째 매개변수로 전달함
그다음 앞에서 직접 검증 세트를 떼어 내지 않고 훈련 세트 전체를 cross_validate()함수에 전달
"""
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00499868, 0.00400209, 0.00499678, 0.00400019, 0.00400209]), 'score_time': array([0.        , 0.00100136, 0.        , 0.0010004 , 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
"""
이 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환함
처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미함
각 키다마 5개의 숫자가 담겨 있습니다
cross_validate()함수는 기본적으로 5-폴드 교차검증을 수행함
cv 매개변수에서 폴드 수를 바꿀 수도 있다

교차검증의 최종 점수는 test_score키에 담긴 5개의 점수를 평균하여 얻을 수 있다
이름은 test_score지만 검증 폴드의 점수임 - 혼동하지 마셈
"""
import numpy as np

print(np.mean(scores["test_score"]))

0.855300214703487


In [9]:
"""
교차 검증을 수행하면 입력한 모델에서 얻을 수 있는 최상의 검증 점수를 가늠해 볼 수 있다

한 가지 주의할 점은 cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않다
앞서 train_test_split()함수로 전체 데이터를 섞은 후 훈련 세틀르 준비 했기 때문에 
따로 섞을 필요가 없는 상황
하지만 만약 교차 검증을 할 때 훈련 세트를 섞으려면 분할기를 지정해야함

사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해 줌
cross_validate() 함수는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용하고 분류 모델일 경우 
타깃 클래스를 골고루 나누기 위해 StratifiedKFold 를 사용함
즉 앞서 수행한 교차 검증은 다음 코드와 동일함
"""
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())

print(np.mean(scores["test_score"]))

0.855300214703487


In [10]:
"""
만약 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면 다음과 같이 작성
"""
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores["test_score"]))

0.8574181117533719


In [11]:
"""
KFold 클래스도 동일한 방식으로 사용할 수 있다
교차 검증에 대해서 이해함

결정트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아보자
이때 테스트 세트를 사용하지 않고 교차 검증을 통해서 좋은 모델을 고르면 됨
"""

'\nKFold 클래스도 동일한 방식으로 사용할 수 있다\n교차 검증에 대해서 이해함\n\n결정트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아보자\n이때 테스트 세트를 사용하지 않고 교차 검증을 통해서 좋은 모델을 고르면 됨\n'

In [12]:
"""
하이퍼파라미터 튜닝

머신러닝 모델이 학습하는 파라미터를 모델 파라미터라고 부름

반면에 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터를 하이퍼파라미터라고 함
사이킷런과 같은 머신러닝 라이브러리를 사용할 때 이런 하이퍼파라미터는 모두 클래스나 메서드의 매개변수로 표현

그럼 이런 하이퍼파라미터를 튜닝하는 작업은 어떻게 진행할까? - 먼저 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련
그다음 검증 세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 바꿔봄
모델마다 적게는 1~2개에서, 많게는 5~6개의 매개변수를 제공함
이 매개변수를 바꿔가면서 모델을 훈련하고 교차 검증을 수행


매개변수를 동시에 바꿔가며 최적의 값을 찾아야 하는 것-

사이킷런에서 제공하는 -그리드서치- 를 사용함

사이킷런의 GridSearchCV클래스는 하이퍼파라미터 탐색과 교차 검증을 한번에 수행함

GridSearchCV클래스를 임포트하고 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만듬
"""
from sklearn.model_selection import GridSearchCV
params = {"min_impurity_decrease" : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [13]:
"""
GridSearchCV 클래스에서 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 생성
"""
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [14]:
"""
그리드 서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바꿔가며 총 5번 실행

GridSearchCV의 cv 매개변수 기본값은 5
따라서 min_impurity_decrease값 마다 5-폴드 교차 검증을 수행함
즉 5X5 = 25 개의 모델을 훈련함
n_jobs 매개변수는 사용할 CPU 코어 수를 지정, -1은 모든 CPU 사용
"""
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [15]:
"""
사이킷런의 그리드 서치는 훈련이 끝나면 25개의 모델 중에서 
검증 점수가 가자 높은 모델의 매개변수 조합으로 
전체 훈련 세트에서 자동으로 다시 모델을 훈련함
best_estimator_ 속성에 저장 되어 있음
"""
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [16]:
"""
그리드 서치로 찾은 최적의 매개변수 
"""
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [17]:
"""
여기서는 0.0001이 가장 좋은 매개변수

각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 mean_test_score 키에 저장됨
5번의 교차 검증으로 얻은 점수를 출력
"""
print(gs.cv_results_["mean_test_score"])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [18]:
"""
첫 번째 값이 가장 커보임, 수동으로 고르는 것보다 넘파이 argmax()함수를 사용하면 가장 큰 값의 인덱스를 추출함
그다음 이 인덱스를 사용해 params 키에 저장된 매개변수를 출력할 수 있다
이 값이 최상의 검증 점수를 만든 매개변수 조합

"""
best_index = np.argmax(gs.cv_results_["mean_test_score"])
print(gs.cv_results_["params"][best_index])

{'min_impurity_decrease': 0.0001}


In [19]:
"""
결정 트리에서 min_impurtiy_decrease는 노드를 분할하기 위한 불순도 감소 최소량을 지정함
여기에다가 max_depth로 트리의 깊이를 제한하고 min_samples_split으로 노드를 나누기 위한 최소 샘플 수도 골라 봄
"""
params = {"min_impurity_decrease": np.arange(0.0001, 0.001, 0.0001),
         "max_depth": range(5, 20, 1),
         "min_samples_split": range(2, 100, 10)}

In [20]:
"""
매개변수로 수행할 교차 검증 횟수는 9 X 15 X 10 = 1350임 
기본 5 - 폴드 교차 검증을 수행하므로 
만들어지는 모델의 수는 6750개 
n_jobs 매개변수를 -1로 설정
"""
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [21]:
"""
최상의 매개변수 조합 확인
"""
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [22]:
"""
최상의 교차 검증 점수 확인
"""
print(np.max(gs.cv_results_["mean_test_score"]))

0.8683865773302731


In [23]:
"""
랜덤 서치

매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려움
또 너무 많은 매개변수 조건이 있어 그리드 서치 수행 기간이 오래 걸릴 수 있다
이럴 때 랜덤 서치를 사용

랜덤 서치에는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 
확률 분포 객체를 전달함 
확률 분포
"""
from scipy.stats import uniform, randint

In [24]:
"""
싸이파이의 stats 서브 패키지에 있는 uniform 과 randint 클래스는 모두 주어진 범위에서 
고르게 값을 뽑습니다 - 이를 "균등 분포에서 샘플링 한다"라고 말함
randint는 정숫값을 뽑고, uniform은 실숫값을 뽑는다
사용하는 방법
"""
rgen = randint(0, 10)
rgen.rvs(10)

array([8, 2, 0, 8, 9, 1, 9, 0, 5, 0])

In [25]:
"""
10개 밖에 되지 않기 때문에 고르게 샘플링 되는것 같지 않지만 샘플링 숫자를 늘리면 
쉽게 확인 가능
"""
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([117,  99,  93, 102, 107,  98, 102,  92,  89, 101], dtype=int64))

In [26]:
"""
uniform 클래스의 사용법도 동일
"""
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.19095159, 0.43890066, 0.79980241, 0.60513978, 0.56419292,
       0.62424485, 0.11681418, 0.67832842, 0.62617246, 0.45094173])

In [27]:
"""
탐색할 매개변수의 딕셔너리를 만들어 봄
여기에는 min_samples_leaf 매개변수를 탐색 대상에 추가함 - 이 매개변수는 리프 노드가 되기 위한 최소 샘플의 개수
어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않다
탐색할 매개변수 범위
"""
params = {"min_impurity_decrease": uniform(0.0001, 0.001),
         "max_depth": randint(20, 50),
         "min_samples_split": randint(2, 25),
         "min_samples_leaf":randint(1, 25),}

In [28]:
"""
샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSerchCV의 n_iter 매개변수에 지정
"""
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                       n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002649F5D4280>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000264DFD89970>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000264BF8A7340>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002649F5C41F0>},
                   random_state=42)

In [29]:
"""
params에 정의된 매개변수 범위에서 총 100번(n_iter 매개변수)을 샘플링하여 
교차 검증을 수행하고 최적의 매개변수 조합을 찾습니다
그리드 서치보다 훨씬 교차 검증 수를 줄이면서 넓은 영역을 효과적으로 탐색할 수 있다
결과를 확인
최적의 매개변수 조합을 출력
"""
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [30]:
"""
최고의 교차 검증 점수도 확인
"""
print(np.max(gs.cv_results_["mean_test_score"]))

0.8695428296438884


In [31]:
"""
최적의 모델은 이미 전체 훈련세트로 훈련되어 best_estimator_ 속성에 저장되어있다

테스트 세트 점수는 검증 세트에 대한 점수보다 조금 작은 것이 일반적
"""
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


In [33]:
"""
확인 문제 
마지막 RandomizedSeacrhV 예제에서 DecisionTreeClassifier 클래스에 splitter = "random" 매개변수를 추가하고 다시 훈련
splitter 매개변수의 기본값은 "best"로 각 노드에서 최선의 분할을 찾습니다
"random"이면 무작위로 분할한 다음 가장 좋은것을 고름
"""
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42, splitter="random"), params,
                       n_iter=100, n_jobs=-1, random_state=42, )
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002649F5D4280>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000264DFD89970>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000264BF8A7340>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002649F5C41F0>},
                   random_state=42)

In [34]:
print(gs.best_params_)

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}


In [35]:
print(np.max(gs.cv_results_["mean_test_score"]))

0.8458726956392981


In [36]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.786923076923077
